**Input**

In [ ]:
day = 16

import pyodide
def read(name):
  return pyodide.open_url('https://raw.githubusercontent.com/joaomoreno/aoc2021/main/%d/%s' % (day, name)).getvalue()

sample = read('sample')
input = read('input')

**Parsing**

In [ ]:
import micropip
await micropip.install('bitstring')
from bitstring import BitStream

def parse(input):
  return [BitStream(bytes.fromhex(line)) for line in input.splitlines(False)]


**Part One**

In [219]:
def read_value(input):
  value = 0

  while True:
    leading = input.read('bool')
    value = (value << 4) | input.read('uint:4')
    if not leading:
      return value

def read_operator_bit_length(input):
  length = input.read('uint:15')
  start = input.pos
  result = []

  while input.pos - start < length:
    result.append(read_packet(input))
  
  return result

def read_operator_packet_length(input):
  packets = input.read('uint:11')

  return [read_packet(input) for _ in range(packets)]

def read_packet(input):
  version = input.read('uint:3')
  type_id = input.read('uint:3')

  if type_id == 4:
    value = read_value(input)
  else:
    if input.read('bool'):
      value = read_operator_packet_length(input)
    else:
      value = read_operator_bit_length(input)
  
  return (version, type_id, value)

def sum_versions(packet):
  queue = [packet]
  result = 0

  while len(queue) > 0:
    version, type_id, value = queue.pop()
    result += version

    if isinstance(value, list):
      queue.extend(value)

  return result

def run(input):
  return [sum_versions(read_packet(line)) for line in input]

run(parse(input))

[947]

**Part Two**

In [218]:
from operator import mul
from functools import reduce

def evaluate(packet):
  _, type_id, value = packet

  if type_id == 0:
    return sum(map(evaluate, value))
  elif type_id == 1:
    return reduce(mul, map(evaluate, value), 1)
  elif type_id == 2:
    return min(map(evaluate, value))
  elif type_id == 3:
    return max(map(evaluate, value))
  elif type_id == 4:
    return value
  elif type_id == 5:
    return 1 if evaluate(value[0]) > evaluate(value[1]) else 0
  elif type_id == 6:
    return 1 if evaluate(value[0]) < evaluate(value[1]) else 0
  elif type_id == 7:
    return 1 if evaluate(value[0]) == evaluate(value[1]) else 0

def run(input):
  return [evaluate(read_packet(line)) for line in input]
  
run(parse(input))

[660797830937]